In [1]:
import os
os.chdir("/Users/users/lourens/RDMP/master_research/code/SGQ_classification/")

# Preprocessing for the CPz catalog
This notebook imports the data from the CPz catalog by Logan+ 2020 from the paper "Unsupervised star, galaxy, QSO classification: Application of HDBSCAN" and generates 5 different datasets each with different sets of colours constructed from the magnitudes listed in the CPz catalog. The 5 different datasets are defined as follows:
- STAR: Contains the colours listed as "*best_star_colours*" in Table 3 of Logan & Fotopoulou (2020).
- GAL: Contains the colours listed as "*best_gal_colours*" in Table 3 of Logan & Fotopoulou (2020).
- QSO: Contains the colours listed as "*best_qso_colours*" in Table 3 of Logan & Fotopoulou (2020).
- ALL: Contains the colours listed in the "ALL" column of Table 2 of Logan & Fotopoulou (2020).
- SDSS: Contains all combinations of the total and 3 arcsecond aperture magnitudes as measured in the ugriz filters by SDSS that are present in the CPz catalog.

Each of these datasets are saved in a separate FITS files which are stored in separate folders corresponding to the dataset type. 

## Imports

In [2]:
import os
from astropy.table import Table

import itertools as iter

from SHARC.utils import insertColors, writeDataset

total_magnitudes = ["umag", "gmag", "rmag", "imag", "zmag",
                    "Ymag", "Jmag", "Hmag", "Kmag",
                    "W1mag", "W2mag"]
aperture_magnitudes = ["u3mag", "g3mag", "r3mag", "i3mag", "Z3mag",
                       "Y3mag", "J3mag", "H3mag", "K3mag"]

# colors used for different classifiers in HDBSCAN paper
colors_STAR = ["Kmag-Y3mag", "Kmag-J3mag", "Kmag-Z3mag", "Kmag-H3mag", "J3mag-K3mag",
               "Y3mag-K3mag", "J3mag-W1mag", "Y3mag-W1mag", "Jmag-Kmag", "H3mag-K3mag",
               "H3mag-W1mag", "Ymag-Kmag", "Hmag-Y3mag", "Y3mag-W2mag", "J3mag-W2mag",
               "imag-g3mag", "Z3mag-W1mag", "Z3mag-K3mag", "zmag-u3mag", "Hmag-J3mag"] # from Table 3 (italics, i.e. colours only)
colors_GAL = ["gmag-Jmag", "Ymag-W1mag", "J3mag-W1mag", "Y3mag-W1mag", "J3mag-W2mag",
              "H3mag-W2mag", "Y3mag-W2mag", "Z3mag-W2mag", "Kmag-J3mag", "H3mag-W1mag",
              "Z3mag-W1mag", "Kmag-H3mag", "Hmag-W2mag", "Kmag-W2mag", "W1mag-W2mag",
              "imag-W2mag", "gmag-Kmag", "gmag-Hmag", "imag-W1mag", "rmag-Hmag",
              "g3mag-i3mag", "rmag-Z3mag", "rmag-imag", "r3mag-i3mag", "K3mag-W2mag",
              "rmag-zmag", "rmag-Y3mag", "Hmag-J3mag", "imag-u3mag"] # from Table 3 (italics, i.e. colours only)
colors_QSO = ["J3mag-W1mag", "Y3mag-W1mag", "J3mag-W2mag", "H3mag-W2mag", "Y3mag-W2mag",
              "Z3mag-W2mag", "Kmag-J3mag", "H3mag-W1mag", "Z3mag-W1mag", "Kmag-H3mag",
              "Hmag-W2mag", "Kmag-W2mag", "W1mag-W2mag", "gmag-Jmag", "imag-W2mag",
              "gmag-Kmag", "gmag-Hmag", "imag-W1mag", "rmag-Hmag", "g3mag-i3mag",
              "rmag-Z3mag", "rmag-imag", "r3mag-i3mag", "K3mag-W2mag", "rmag-zmag",
              "rmag-Y3mag", "Hmag-J3mag", "imag-u3mag"] # from Table 3 (italics, i.e. colours only)
colors_ALL = ["Kmag-Y3mag", "Kmag-J3mag", "Kmag-H3mag", "J3mag-W1mag", "J3mag-K3mag",
              "Y3mag-W1mag", "H3mag-W1mag", "H3mag-K3mag", "Jmag-Kmag", "Y3mag-K3mag"] # from Table 2 ("ALL")
colors_SDSS = [m1 + "-" + m2 for m1, m2 in iter.combinations(total_magnitudes[:5] + aperture_magnitudes[:5], 2)]

# path to storage
data_path = "/Users/users/lourens/RDMP/master_research/data/HDBSCAN2"

# import CPz dataset
table = Table.read(os.path.join(data_path, "cpz.dat"), readme=os.path.join(data_path, "ReadMe"), format="ascii.cds")
display(table)

id,RAdeg,DEdeg,z,Hclass,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Ymag,e_Ymag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,W1mag,e_W1mag,W2mag,e_W2mag,u3mag,e_u3mag,g3mag,e_g3mag,r3mag,e_r3mag,i3mag,e_i3mag,Z3mag,e_Z3mag,Y3mag,e_Y3mag,J3mag,e_J3mag,H3mag,e_H3mag,K3mag,e_K3mag,Yhlr,Jhlr,Hhlr,Khlr,PCAs1c,PCAs2c,PCAs3c,PCAg1c,PCAg2c,PCAg3c,PCAq1c,PCAq2c,PCAq3c,ClasscO,CassscA,dpc,Poutc,Pstarc,Pgalc,Pqsoc,Labelc,PCAs1cHLR,PCAs2cHLR,PCAs3cHLR,PCAg1cHLR,PCAg2cHLR,PCAg3cHLR,PCAq1cHLR,PCAq2cHLR,PCAq3cHLR,ClasscHLRO,ClasscHLRA,dpcHLR,PoutcHLR,PstarcHLR,PgalcHLR,PqsocHLR,LabelcHLR
,deg,deg,,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,arcsec,arcsec,arcsec,arcsec,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str24,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,int64
0104-00119,34.41787,-4.72127,0.72559,1,23.4122,0.0505,22.8534,0.0211,22.2006,0.0206,21.3921,0.0125,20.9957,0.0205,20.942,0.1872,21.307,0.8141,20.6116,0.4225,20.0746,0.2015,19.2583,0.071,20.1558,0.305,23.6573,0.0413,23.1494,0.0175,22.5104,0.017,21.6177,0.0097,21.3693,0.0174,20.9169,0.1349,21.2552,0.6533,20.4513,0.3581,19.7707,0.1487,0.75589,0.64208,0.66781,0.62961,-1.15732,-2.11252,0.26002,-1.11161,-0.27282,-0.44797,-1.06384,-0.26909,-0.42575,2,2,0,0.0,0.0,1.0,0.0,2,-1.45773,-0.82213,0.18082,-1.4632,-0.16978,0.00834,0.97244,-0.09133,-0.75841,2,2,0,0.0,0.0,0.99,0.01,2
0104-00184,34.37192,-4.59308,0.44925,1,22.018,0.0229,21.3995,0.0088,20.4452,0.0065,19.9624,0.0057,19.8319,0.0107,19.6647,0.0713,19.6658,0.2254,19.1885,0.1735,19.0394,0.1412,19.3925,0.08,19.8699,0.244,22.5703,0.0175,21.9737,0.0065,21.0491,0.005,20.5636,0.0044,20.4156,0.0074,19.931,0.0653,19.8437,0.1613,19.5163,0.1552,19.3704,0.1149,1.04066,0.92622,1.00097,1.0041,-0.19838,-0.40535,0.55346,-0.03172,-0.20366,0.13045,-0.02409,-0.20295,0.13825,2,2,0,0.0,0.0,1.0,0.0,2,-0.02468,0.10831,0.1801,-0.03298,0.12811,0.17348,0.07296,0.04991,-0.26422,2,2,0,0.0,0.0,1.0,0.0,2
0104-00227,34.41814,-4.5809,0.699164,1,22.4891,0.0389,21.7992,0.0141,21.0215,0.012,20.0603,0.0068,19.7914,0.0115,19.5855,0.0688,19.35,0.1776,19.0366,0.1412,18.6493,0.0827,18.4685,0.047,19.0239,0.12,23.336,0.0328,22.562,0.0109,21.7882,0.0091,20.8264,0.0053,20.5733,0.0084,19.9825,0.0631,19.616,0.1312,19.251,0.122,18.9551,0.0708,0.971,0.95445,0.98895,0.94643,-0.93719,-0.72131,0.39908,-1.03261,-0.04297,-0.12212,-1.02196,-0.04914,-0.13224,2,2,0,0.0,0.0,1.0,0.0,2,-1.0812,-0.7045,0.3711,-1.13309,-0.3724,0.2693,0.95536,0.1676,-0.21571,2,2,0,0.0,0.0,1.0,0.0,2
0104-00250,34.28431,-4.5688,0.700158,1,25.1815,0.2072,24.5456,0.0827,23.2131,0.0465,22.047,0.0197,21.5288,0.0272,21.3728,0.2525,21.0723,0.5306,20.5104,0.4139,20.1193,0.2532,19.6805,0.1,20.3989,0.407,25.5502,0.2235,24.7088,0.0728,23.4384,0.0425,22.2685,0.0179,21.7897,0.0253,21.2665,0.2362,20.9326,0.4821,20.4252,0.327,20.1081,0.2153,0.62192,0.66663,0.66207,0.61048,-0.9556,-1.09699,-0.55277,-1.43977,0.88886,-0.79804,-1.41286,0.88817,-0.79365,2,2,0,0.04,0.0,0.96,0.0,2,-1.33912,-1.57864,-0.21313,-1.58641,-1.1678,-0.39228,1.6379,0.78371,-0.66597,2,2,0,0.0,0.0,1.0,0.0,2
0104-00318,34.41644,-4.45668,0.8911,1,23.408,0.0784,23.1214,0.0391,22.5987,0.045,21.7409,0.0255,21.3541,0.038,20.9172,0.2132,21.0162,0.6402,20.3631,0.3498,20.0327,0.2235,19.4532,0.09,20.4447,0.441,23.6476,0.0552,23.3975,0.0277,22.8581,0.0311,22.036,0.0179,21.6268,0.0255,2

## Write datasets

In [3]:
datasets = ["STAR", "GAL", "QSO", "ALL", "SDSS"]
for dataset in datasets:
    table_tmp = table.copy() # make a hardcopy of the table
    insertColors(table_tmp, eval(f"colors_{dataset}"))
    writeDataset(table_tmp, filename=os.path.join(data_path, dataset, f"CPz_{dataset}_dataset.fits.gz"), verbose=True, overwrite=True)
    del table_tmp

Writing data to /Users/users/lourens/RDMP/master_research/data/HDBSCAN2/STAR/CPz_STAR_dataset.fits.gz ...
Write successful!
Writing data to /Users/users/lourens/RDMP/master_research/data/HDBSCAN2/GAL/CPz_GAL_dataset.fits.gz ...
Write successful!
Writing data to /Users/users/lourens/RDMP/master_research/data/HDBSCAN2/QSO/CPz_QSO_dataset.fits.gz ...
Write successful!
Writing data to /Users/users/lourens/RDMP/master_research/data/HDBSCAN2/ALL/CPz_ALL_dataset.fits.gz ...
Write successful!
Writing data to /Users/users/lourens/RDMP/master_research/data/HDBSCAN2/SDSS/CPz_SDSS_dataset.fits.gz ...
Write successful!
